# 세이지메이커 LMI와 Rolling Batch를 활용해 세이지메이커에서 높은 성능으로 Llama2 70B 모델 배포

이 노트북에서는 DeepSpeed를 활용해 세이지메이커에서 FP16 정밀도로 Llama2 대규모 언어 모델을 호스팅하는 방법을 살펴보겠습니다. 이 예시에서는 LMI 컨테이너에 포함된 DJLServing을 모델 서빙 솔루션으로 사용합니다. DJLServing은 Deep Java Library (DJL)를 기반으로 하는 고성능의 범용 모델 서빙 솔루션으로, 프로그래밍 언어에 구애받지 않습니다. DJL과 DJLServing에 대한 더 자세한 내용은 최근 블로그 포스트에서 확인할 수 있습니다(https://aws.amazon.com/blogs/machine-learning/deploy-bloom-176b-and-opt-30b-on-amazon-sagemaker-with-large-model-inference-deep-learning-containers-and-deepspeed/).

모델 병렬화는 단일 GPU의 용량을 초과하는 대규모 모델을 배포하는 데 도움이 될 수 있습니다. 모델 병렬화를 활용하면 모델을 여러 GPU에 분할해 분산시킵니다. 각 GPU는 모델의 서로 다른 부분을 보유하므로, 수십억 개의 매개변수를 가진 가장 큰 딥러닝 모델의 메모리 용량 문제를 해결합니다.

세이지메이커는 이제 DeepSpeed 컨테이너를 제공해, 사용자가 관리형 서빙 기능을 활용하고 비즈니스에 차별화되지 않은 무거운 작업을 처리할 수 있게 합니다.

이 노트북에서는 https://huggingface.co/TheBloke/Llama-2-70b-fp16 모델을 ml.g5.48xlarge 인스턴스에 배포합니다.

# 라이센스 계약
 - 모델을 사용하기 전에 [여기](https://huggingface.co/meta-llama)에서 라이선스 정보를 확인하기 바랍니다.
 - 이 노트북은 샘플 노트북이며 운영 용도가 사용하기에 적합하지 않습니다. 라이센스는 [여기](https://github.com/aws/mit-0)에서 확인할 수 있습니다.

In [2]:
!pip install sagemaker boto3 --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 41.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 1.2 MB/s eta 0:00:000:00:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.33.9
    Uninstalling botocore-1.33.9:
      Successfully uninstalled botocore-1.33.9
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.8.2
    Uninstalling s3transfer-0.8.2:
      Successfully uninstalled s3transfer-0.8.2
  Attempting uninstall: boto3
    Found existing installation: boto3 1.33.9
    Uninstalling boto3-1.33.9:
      Successfully uninstalled boto3-1.33.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.31.9 requires botocore==1.33.9, but you have botocore 1.34.19 which is

In [3]:
import sagemaker
import jinja2
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
role = sagemaker.get_execution_role()  # 엔드포인트 실행 역할
sess = sagemaker.session.Session()  # 다양한 AWS API와 상호작용하기 위한 세이지메이커 세션
bucket = sess.default_bucket()  # 아티팩트를 저장할 버킷

In [23]:
model_bucket = sess.default_bucket()  # 아티팩트를 저장할 버킷
s3_code_prefix = "hf-large-model-djl/meta-llama/Llama-2-70b-fp16/code"  # 코드 아티팩트가 들어갈 버킷 내 폴더

s3_model_prefix = "hf-large-model-djl/meta-llama/Llama-2-70b-fp16/model"  # 코드 아티팩트가 들어갈 버킷 내 폴더
region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sagemaker_client = boto3.client("sagemaker")
sagemaker_runtime_client = boto3.client("sagemaker-runtime")

jinja_env = jinja2.Environment()

### 모델이 위치한 S3 URL을 포함하는 변수를 정의합니다.

In [9]:
# 모델이 위치한 S3 URL을 포함하는 변수를 정의합니다. 데모 목적으로, S3 버킷에서 Llama-2-70b-fp16 모델 아티팩트를 활용합니다.
pretrained_model_location = f"s3://sagemaker-example-files-prod-{region}/models/llama-2/fp16/70B/"

## 세이지메이커 호환 모델 아티팩트 생성하고, 모델을 S3에 업로드한 후 자체 추론 스크립트 가져오기

세이지메이커 대규모 모델 추론 컨테이너는 자체 추론 코드를 제공하지 않고도 모델을 호스팅하는 데 활용할 수 있습니다. 이는 입력 데이터의 사용자 정의 전처리나 모델의 예측 결과에 대한 후처리가 필요 없는 경우에 매우 유용합니다.

세이지메이커는 모델 아티팩트가 Tarball 형식이어야 합니다. 이 예시에서는 serving.properties 파일을 제공합니다.
tarball은 다음과 같은 형식입니다.

```
code
├────
│   └── serving.properties
```

    serving.properties는 모델 서버 구성 파일입니다.


#### serving.properties 생성
이 파일은 DJL Serving에 활용할 모델 병렬화 및 추론 최적화 라이브러리를 지정하는 구성 파일입니다. 필요에 따라 적절한 구성을 설정할 수 있습니다.

이 구성 파일의 설정 목록은 다음과 같습니다.

    engine: DJL이 활용할 엔진을 지정합니다. 여기서는 MPI로 설정했습니다.
    option.model_id: 사전 학습된 모델의 ID를 지정합니다. 이는 huggingface.co의 모델 저장소에 호스팅된 모델 ID이거나 모델 아티팩트의 S3 경로일 수 있습니다.
    option.tensor_parallel_degree: 모델을 분할하는 데 필요한 GPU 장치의 수를 설정합니다. 이 매개변수는 DJL serving이 실행될 때 시작되는 모델당 워커 수를 제어합니다. 예를 들어, 4개의 GPU 머신에서 4개의 파티션을 생성하는 경우, 요청을 처리하기 위해 모델당 1개의 워커가 시작됩니다.

구성 옵션 및 전체 목록에 대한 자세한 내용은 [설명서](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html)에서 확인할 수 있습니다.

In [10]:
!rm -rf code_llama2_70b_fp16
!mkdir -p code_llama2_70b_fp16

In [11]:
%%writefile code_llama2_70b_fp16/serving.properties
engine = MPI
option.tensor_parallel_degree = 8
option.rolling_batch = auto
option.max_rolling_batch_size = 4
option.model_loading_timeout = 3600
option.model_id = {{model_id}}
option.paged_attention = true
option.trust_remote_code = true
option.dtype = fp16

Writing code_llama2_70b_fp16/serving.properties


In [12]:
# `serving.properties` 파일에 적절한 모델 위치를 설정합니다.
template = jinja_env.from_string(Path("code_llama2_70b_fp16/serving.properties").open().read())
Path("code_llama2_70b_fp16/serving.properties").open("w").write(
    template.render(model_id=pretrained_model_location)
)
!pygmentize code_llama2_70b_fp16/serving.properties | cat -n

     1	engine = MPI
     2	option.tensor_parallel_degree = 8
     3	option.rolling_batch = auto
     4	option.max_rolling_batch_size = 4
     5	option.model_loading_timeout = 3600
     6	option.model_id = s3://sagemaker-example-files-prod-us-east-1/models/llama-2/fp16/70B/
     7	option.paged_attention = true
     8	option.trust_remote_code = true
     9	option.dtype = fp16


**DJL 컨테이너의 이미지 URI를 활용합니다.**

In [13]:
inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed", region=region, version="0.23.0"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


**Tarball을 생성한 후 S3 위치에 업로드합니다.**

In [14]:
!rm model.tar.gz
!tar czvf model.tar.gz code_llama2_70b_fp16

rm: cannot remove 'model.tar.gz': No such file or directory
code_llama2_70b_fp16/
code_llama2_70b_fp16/serving.properties


In [15]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)

### 엔드포인트를 생성하는 단계는 다음과 같습니다.

1. 이미지 컨테이너와 이전에 업로드한 모델 Tarball을 활용해 모델을 생성합니다.
2. 다음 주요 매개변수를 사용해 엔드포인트 구성을 생성합니다.

    a) 인스턴스 유형은 `ml.g5.48xlarge`입니다.

    b) `ContainerStartupHealthCheckTimeoutInSeconds`는 3600으로 설정해 모델이 준비된 후 헬스 체크가 시작되도록 합니다.
3. 생성한 엔드포인트 구성으로 엔드포인트를 생성합니다.

#### 모델 생성
DJL 컨테이너의 이미지 URI와 Tarball이 업로드된 S3 위치를 활용합니다.

컨테이너는 모델을 인스턴스의 `/tmp` 공간에 다운로드합니다. 세이지메이커는 `/tmp`를 아마존 EBS(Amazon Elastic Block Store) 볼륨에 매핑하며, 이 볼륨은 엔드포인트 생성 매개변수 `VolumeSizeInGB`를 지정할 때 마운트됩니다. 컨테이너는 매우 빠른 다운로드 속도를 제공하는 `s5cmd`(https://github.com/peak/s5cmd)를 활용해 대규모 모델 다운로드에 유용합니다.

p4dn과 같이 볼륨 인스턴스가 사전 구축된 경우, 컨테이너의 `/tmp`를 계속 활용할 수 있습니다. 이 마운트의 크기는 모델을 저장하기에 충분히 큽니다.

In [16]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"Llama-2-70b-fp16-mpi")
print(model_name)

create_model_response = sagemaker_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact,
        "Environment": {"MODEL_LOADING_TIMEOUT": "3600"},
    },
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

Llama-2-70b-fp16-mpi-2024-01-16-04-32-58-391
Created Model: arn:aws:sagemaker:us-east-1:079002598131:model/llama-2-70b-fp16-mpi-2024-01-16-04-32-58-391


In [17]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.48xlarge",
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": 3600,
            "ContainerStartupHealthCheckTimeoutInSeconds": 3600,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:079002598131:endpoint-config/llama-2-70b-fp16-mpi-2024-01-16-04-32-58-391-config',
 'ResponseMetadata': {'RequestId': '020920e1-7acb-4249-a892-b10ed85ad12b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '020920e1-7acb-4249-a892-b10ed85ad12b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '132',
   'date': 'Tue, 16 Jan 2024 04:32:59 GMT'},
  'RetryAttempts': 0}}

In [19]:
create_endpoint_response = sagemaker_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-1:079002598131:endpoint/llama-2-70b-fp16-mpi-2024-01-16-04-32-58-391-endpoint


In [20]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(
            region, endpoint_name
        )
    )
)

/tmp/ipykernel_1713/4280955313.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### 이 단계는 약 20분 이상 소요될 수 있으니, 인내심을 가지고 기다려 주시기 바랍니다.

In [21]:
import time

resp = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-1:079002598131:endpoint/llama-2-70b-fp16-mpi-2024-01-16-04-32-58-391-endpoint
Status: InService


#### 엔드포인트가 생성될 때까지 기다리는 동안, 다음 내용을 더 읽어볼 수 있습니다.
- [대규모 모델 추론을 위한 딥러닝 컨테이너](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-dlc.html)

#### Boto3를 활용해 엔드포인트 호출

이 모델은 생성 모델이므로, 프롬프트로 텍스트를 전달하면 모델이 문장을 완성하고 결과를 반환합니다.

모델에 프롬프트를 입력으로 전달할 수 있습니다. 이는 inputs를 프롬프트로 설정해 수행됩니다. 그러면 모델은 각 프롬프트에 대한 결과를 반환합니다. 텍스트 생성은 적절한 매개변수를 활용해 구성할 수 있습니다. 이러한 매개변수는 kwargs 사전으로 엔드포인트에 전달해야 합니다. 자세한 내용은 [문서](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig)에서 확인할 수 있습니다.

아래 코드 샘플은 텍스트 프롬프트를 활용해 엔드포인트를 호출하고 몇 가지 매개변수를 설정하는 방법을 보여줍니다.

In [ ]:
sagemaker_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": "The diamondback terrapin was the first reptile to do what?",
            "parameters": {
                "do_sample": True,
                "max_new_tokens": 100,
                "min_new_tokens": 100,
                "temperature": 0.3,
                "watermark": True,
            },
        }
    ),
    ContentType="application/json",
)["Body"].read().decode("utf8")

# 세이지메이커 엔드포인트 오토스케일링

In [29]:
autoscale = boto3.Session().client(service_name="application-autoscaling")

In [31]:
autoscale.register_scalable_target(
    ServiceNamespace="sagemaker",
    ResourceId="endpoint/" + endpoint_name + "/variant/variant1",
    ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    MinCapacity=1,
    MaxCapacity=2,
    RoleARN=role,
    SuspendedState={
        "DynamicScalingInSuspended": False,
        "DynamicScalingOutSuspended": False,
        "ScheduledScalingSuspended": False,
    },
)

{'ScalableTargetARN': 'arn:aws:application-autoscaling:us-east-1:079002598131:scalable-target/056ma51e31786bfc49aca03c66fe0feb28f9',
 'ResponseMetadata': {'RequestId': '8aecb2e7-b22f-4ea0-8289-9f55d53b4856',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8aecb2e7-b22f-4ea0-8289-9f55d53b4856',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '131',
   'date': 'Tue, 16 Jan 2024 04:47:09 GMT'},
  'RetryAttempts': 0}}

In [32]:
# 대상이 사용 가능한지 확인합니다.
autoscale.describe_scalable_targets(
    ServiceNamespace="sagemaker",
    MaxResults=100,
)

{'ScalableTargets': [{'ServiceNamespace': 'sagemaker',
   'ResourceId': 'endpoint/Llama-2-70b-fp16-mpi-2024-01-16-04-32-58-391-endpoint/variant/variant1',
   'ScalableDimension': 'sagemaker:variant:DesiredInstanceCount',
   'MinCapacity': 1,
   'MaxCapacity': 2,
   'RoleARN': 'arn:aws:iam::079002598131:role/aws-service-role/sagemaker.application-autoscaling.amazonaws.com/AWSServiceRoleForApplicationAutoScaling_SageMakerEndpoint',
   'CreationTime': datetime.datetime(2024, 1, 16, 4, 47, 9, 459000, tzinfo=tzlocal()),
   'SuspendedState': {'DynamicScalingInSuspended': False,
    'DynamicScalingOutSuspended': False,
    'ScheduledScalingSuspended': False},
   'ScalableTargetARN': 'arn:aws:application-autoscaling:us-east-1:079002598131:scalable-target/056ma51e31786bfc49aca03c66fe0feb28f9'}],
 'ResponseMetadata': {'RequestId': '53321fdb-1cb2-4876-aad9-bfa67eaf3753',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '53321fdb-1cb2-4876-aad9-bfa67eaf3753',
   'content-type': 'appl

In [33]:
autoscale.put_scaling_policy(
    PolicyName="autoscale-policy-gpu-400-llama2-70b",
    ServiceNamespace="sagemaker",
    ResourceId="endpoint/" + endpoint_name + "/variant/variant1",
    ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    PolicyType="TargetTrackingScaling",
    TargetTrackingScalingPolicyConfiguration={
        "TargetValue": 400, # 400% of 800% total GPU utilization (8 GPUs)
        "CustomizedMetricSpecification":
        {
            "MetricName": "GPUUtilization",
            "Namespace": "/aws/sagemaker/Endpoints",
            "Dimensions": [
                {"Name": "EndpointName", "Value": endpoint_name},
                {"Name": "VariantName", "Value": "variant1"}
            ],
            "Statistic": "Average",
            "Unit": "Percent"
        },
        "ScaleOutCooldown": 60,
        "ScaleInCooldown": 300,
    }
)

{'PolicyARN': 'arn:aws:autoscaling:us-east-1:079002598131:scalingPolicy:a51e3178-6bfc-49ac-a03c-66fe0feb28f9:resource/sagemaker/endpoint/Llama-2-70b-fp16-mpi-2024-01-16-04-32-58-391-endpoint/variant/variant1:policyName/autoscale-policy-gpu-400-llama2-70b',
 'Alarms': [{'AlarmName': 'TargetTracking-endpoint/Llama-2-70b-fp16-mpi-2024-01-16-04-32-58-391-endpoint/variant/variant1-AlarmHigh-b34fa515-574f-4f65-84f3-627a58040aa9',
   'AlarmARN': 'arn:aws:cloudwatch:us-east-1:079002598131:alarm:TargetTracking-endpoint/Llama-2-70b-fp16-mpi-2024-01-16-04-32-58-391-endpoint/variant/variant1-AlarmHigh-b34fa515-574f-4f65-84f3-627a58040aa9'},
  {'AlarmName': 'TargetTracking-endpoint/Llama-2-70b-fp16-mpi-2024-01-16-04-32-58-391-endpoint/variant/variant1-AlarmLow-cb39c7ac-448f-4a05-bb7b-3333c57acc02',
   'AlarmARN': 'arn:aws:cloudwatch:us-east-1:079002598131:alarm:TargetTracking-endpoint/Llama-2-70b-fp16-mpi-2024-01-16-04-32-58-391-endpoint/variant/variant1-AlarmLow-cb39c7ac-448f-4a05-bb7b-3333c57acc0

# 오토스케일링 실행

In [ ]:
for i in range(0, 100):
    res = sagemaker_runtime_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(
            {
                "inputs": "The diamondback terrapin was the first reptile to do what?",
                "parameters": {
                    "do_sample": True,
                    "max_new_tokens": 100,
                    "min_new_tokens": 100,
                    "temperature": 0.3,
                    "watermark": True,
                },
            }
        ),
        ContentType="application/json",
    )
    print(f'{i}: {res["Body"].read().decode("utf8")}')

In [37]:
autoscale.describe_scaling_activities(
    ServiceNamespace="sagemaker",
    ResourceId="endpoint/" + endpoint_name + "/variant/variant1",
    ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    MaxResults=100
)

{'ScalingActivities': [{'ActivityId': 'f623aaf1-a0c8-4662-a513-e7860a90153a',
   'ServiceNamespace': 'sagemaker',
   'ResourceId': 'endpoint/Llama-2-70b-fp16-mpi-2024-01-16-04-32-58-391-endpoint/variant/variant1',
   'ScalableDimension': 'sagemaker:variant:DesiredInstanceCount',
   'Description': 'Setting desired instance count to 2.',
   'Cause': 'monitor alarm TargetTracking-endpoint/Llama-2-70b-fp16-mpi-2024-01-16-04-32-58-391-endpoint/variant/variant1-AlarmHigh-b34fa515-574f-4f65-84f3-627a58040aa9 in state ALARM triggered policy autoscale-policy-gpu-400-llama2-70b',
   'StartTime': datetime.datetime(2024, 1, 16, 4, 51, 28, 298000, tzinfo=tzlocal()),
   'StatusCode': 'InProgress',
   'StatusMessage': 'Successfully set desired instance count to 2. Waiting for change to be fulfilled by sagemaker.'}],
 'ResponseMetadata': {'RequestId': 'ccba093d-51d6-4527-9f31-f53fe75411ee',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ccba093d-51d6-4527-9f31-f53fe75411ee',
   'conte

## 정리

In [ ]:
# # - 엔드 포인트 제거
# sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
# # - 엔드포인트가 실패할 경우, 모델을 삭제하는 것도 고려해야 합니다.
# sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
# sagemaker_client.delete_model(ModelName=model_name)